# DACON 재정정보 AI 검색 알고리즘 경진대회
- - -
태스크: QA  
모델: RAG  
&emsp; Generator: *rtzr/ko-gemma-2-9b-it*  
&emsp;&emsp; 학습: Lora(학습 후 성능 저하 발견. train 데이터 전처리 문제)  
&emsp; Retriever: *BAAI/bge-m3* (base), *BAAI/bge-reranker-v2-m3* (reranker)  
DB: single vs multi vector  
&emsp; 테스트 데이터의 source 파일을 참고 안하기 vs 하기

[dacon link](
https://dacon.io/competitions/official/236295/overview/description)

# 0. Setting

In [1]:
!python --version

Python 3.10.12


In [2]:
!pip install -qU transformers sentence-transformers datasets bitsandbytes
!pip install -qU langchain chromadb langchain-chroma langchain_community faiss-gpu
!pip install -q peft
# !pip install pymupdf4llm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4

In [ ]:
# !pip install --quiet langchain_experimental langchain_openai

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# import pymupdf4llm

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers import ContextualCompressionRetriever

from langchain_chroma import Chroma

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

import re
from glob import glob

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [5]:

project_path = '/content/drive/MyDrive/Data/dacon/finance'
data_path_test = project_path + '/open/test_source'
image_path  = project_path + '/processed/image'

data_file_list_test = glob(data_path_test + '/*')
data_file_list_test

['/content/drive/MyDrive/Data/dacon/finance/open/test_source/보건복지부_부모급여(영아수당) 지원.pdf',
 '/content/drive/MyDrive/Data/dacon/finance/open/test_source/산업통상자원부_에너지바우처.pdf',
 '/content/drive/MyDrive/Data/dacon/finance/open/test_source/보건복지부_노인장기요양보험 사업운영.pdf',
 '/content/drive/MyDrive/Data/dacon/finance/open/test_source/국토교통부_행복주택출자.pdf',
 '/content/drive/MyDrive/Data/dacon/finance/open/test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf',
 '/content/drive/MyDrive/Data/dacon/finance/open/test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》 v2.pdf',
 '/content/drive/MyDrive/Data/dacon/finance/open/test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》 v2.pdf',
 '/content/drive/MyDrive/Data/dacon/finance/open/test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》 v2.pdf',
 '/content/drive/MyDrive/Data/dacon/finance/open/test_source/「FIS 이슈 & 포커스」 23-2호 《핵심

# 1. Load PDF(pymupdf4llm)


In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1600, chunk_overlap=50, separators=["\n\n-----\n", "\n\n", "\n", " ", ""]) # 한 페이지를 가득 채우는 table 하나가 1300~1400 정도

## 1.1 use pdfloader *pymupdf4llm*

In [43]:
documents_test = []

for data_file in data_file_list_test:
    doc_name = data_file.split("/")[-1].replace(".pdf", "").strip()
    print(doc_name)
    pdf = pymupdf4llm.to_markdown(data_file) # , write_images=True, image_path=project_path+'/processed/image/pymupdf4llm/' 이미지 나중에
    pdf =  re.sub(r'(?<!\n)\n\n(?!\n)', '\n', pdf) # 약간의 전처리
    documents_test.append(Document(page_content=pdf, metadata={"title": doc_name}))

split_documents_test = text_splitter.split_documents(documents_test)

보건복지부_부모급여(영아수당) 지원
Processing /content/drive/MyDrive/Data/dacon/finance/open/test_source/보건복지부_부모급여(영아수당) 지원.pdf...
[                                        ] (0/3)=============[=============                           ] (1/3)=============[==========================              ] (2/3)==============[========================================] (3/3)]
산업통상자원부_에너지바우처
Processing /content/drive/MyDrive/Data/dacon/finance/open/test_source/산업통상자원부_에너지바우처.pdf...
[                                        ] (0/11)===[===                                     ] ( 1/11)

## 1.2 load processed data

In [29]:
import pickle

split_documents_test2 = project_path +'/tot_documents.pkl'
# 데이터를 불러오기
with open(split_documents_test2, 'rb') as f:
    split_documents_test2 = pickle.load(f)

# 2. Make DB

In [59]:
base_directory = '/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/'

In [54]:
model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
bge_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

<ipython-input-54-e517834dc675>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  bge_embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

## 2.1 single vector db

In [60]:
vectordb_bge = Chroma.from_documents(
    documents=split_documents_test , #split_documents_test2
    embedding=bge_embeddings,
    persist_directory=base_directory+'single_db_test/1'
)

## 2.2 multi vector db

In [65]:
import pickle
import os
from tqdm import tqdm
import unicodedata
import shutil

key = [
    '국토교통부_행복주택출자',
    '보건복지부_노인장기요양보험 사업운영',
    '보건복지부_부모급여(영아수당) 지원',
    '산업통상자원부_에너지바우처',
    '중소벤처기업부_혁신창업사업화자금(융자)',
    '중앙-지방 간 재정조정제도',
    '우발부채',
    '재정성과관리제도',
    '핵심재정사업 성과관리'
]

def create_vector_db(document, embeddings, file_path):
    db = Chroma.from_documents(
        documents=document,
        embedding=embeddings, persist_directory=file_path)
    return db

def create_pdf_databases(base_directory, splited_docs, embeddings, flag=True):
    """딕셔너리에 pdf명을 키로 해서 DB, retriever 저장"""
    pdf_databases = {}
    split_documents_test_ = {}

    for doc in split_documents_test:
        doc_name = doc.metadata['title'].split(" > ")[0].strip()
        split_documents_test_.setdefault(doc_name, []).append(doc)

    for p, docs in tqdm(split_documents_test_.items(), desc="Creating Vectordb for each PDF"):
        # vectordb의 저장 경로 설정
        vectordb_path = base_directory + p
        # print(vectordb_path)

        # 경로에 파일이 이미 존재하는지 확인
        if os.path.exists(vectordb_path):
            if flag:
                print(f"Vector DB already exists at {vectordb_path}, delete existing DB...")
                shutil.rmtree(vectordb_path)
                print(f"Creating new Vector DB at {vectordb_path}...")
                vectordb = create_vector_db(docs, embeddings, vectordb_path)
            else:
                print(f"Vector DB already exists at {vectordb_path}, loading existing DB...")
                vectordb = Chroma(persist_directory=vectordb_path, embedding_function=embeddings)
        else:
            print(f"Creating new Vector DB at {vectordb_path}...")
            vectordb = create_vector_db(docs, embeddings, vectordb_path)

        # retriever = vectordb.as_retriever(search_kwargs={"k": 3})
        pdf_databases[unicodedata.normalize('NFC', p)] = {
            'db': vectordb,
            # 'retriever': retriever,
            'doc': docs
        }

    return pdf_databases

pdf_databases = create_pdf_databases(base_directory+'multi_db_test/1/', split_documents_test, bge_embeddings) # split_documents_test2

Creating Vectordb for each PDF:   0%|          | 0/9 [00:00<?, ?it/s]

Creating new Vector DB at /content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/multi_db_test/1/보건복지부_부모급여(영아수당) 지원...


Creating Vectordb for each PDF:  11%|█         | 1/9 [00:00<00:06,  1.18it/s]

Creating new Vector DB at /content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/multi_db_test/1/산업통상자원부_에너지바우처...


Creating Vectordb for each PDF:  22%|██▏       | 2/9 [00:04<00:17,  2.45s/it]

...

Creating Vectordb for each PDF:  78%|███████▊  | 7/9 [00:14<00:04,  2.41s/it]

Creating new Vector DB at /content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/multi_db_test/1/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》 v2...


Creating Vectordb for each PDF:  89%|████████▉ | 8/9 [00:18<00:03,  3.01s/it]

Creating new Vector DB at /content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/multi_db_test/1/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》 v2...


Creating Vectordb for each PDF: 100%|██████████| 9/9 [00:23<00:00,  2.61s/it]


In [66]:
%cd /content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/multi_db_test/1
!pwd
!ls -al

/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/multi_db_test/1
/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/multi_db_test/1
total 36
drwx------ 3 root root 4096 Sep 18 06:20 '「FIS 이슈&포커스」 22-2호 《재정성과관리제도》 v2'
drwx------ 3 root root 4096 Sep 18 06:20 '「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》 v2'
drwx------ 3 root root 4096 Sep 18 06:20 '「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》 v2'
drwx------ 3 root root 4096 Sep 18 06:20 '「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》 v2'
drwx------ 3 root root 4096 Sep 18 06:19  국토교통부_행복주택출자
drwx------ 3 root root 4096 Sep 18 06:19 '보건복지부_노인장기요양보험 사업운영'
drwx------ 3 root root 4096 Sep 18 06:19 '보건복지부_부모급여(영아수당) 지원'
drwx------ 3 root root 4096 Sep 18 06:19  산업통상자원부_에너지바우처
drwx------ 3 root root 4096 Sep 18 06:20 '중소벤처기업부_혁신창업사업화자금(융자)'


# 3. load DB

In [6]:
# (1, 2), (3) 둘 중 하나
model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
bge_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

<ipython-input-6-a3648fa44934>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  bge_embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

## 3.1 single vector db

In [7]:
vectordb_bge = Chroma(persist_directory='/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/single_db_test/1', embedding_function=bge_embeddings)

## 3.2 multi vector db

In [12]:
import unicodedata

persist_directory = '/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/multi_db_test/1'
pdf_databases_path = glob(persist_directory + '/*')

pdf_databases_ = {}

for db in pdf_databases_path:
    db_name = db.split('/')[-1]

    pdf_databases_[unicodedata.normalize('NFC', db_name)] = {
            'db': Chroma(persist_directory=db, embedding_function=bge_embeddings)
        }

# 4. Make retriever

## 4.1 single vector db

In [8]:
retriever = vectordb_bge.as_retriever(search_kwargs={"k": 3}) # search_type="similarity_score_threshold", search_kwargs={"k": 3, 'score_threshold': 0.5}

In [11]:
docs = retriever.get_relevant_documents("2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?")

for d in docs:
    print(d.metadata)
    print(d.page_content)
    print("\n=========\n\n")

{'title': '중소벤처기업부_혁신창업사업화자금(융자)'}
## 사  업  명
 혁신창업사업화자금(융자) (5152-301)


## 1. 사업 코드 정보
|구분|기금|소관|실국(기관)|계정|분야|부문|
|---|---|---|---|---|---|---|
|코드|중소벤처기업 창업 및 진흥기금|중소벤처기업부|중소기업정책실||110|118|
|명칭|||||산업·중소기업 및 에너지|창업 및 벤처|


|구분|프로그램|단위사업|세부사업|
|---|---|---|---|
|코드|5100|5152|301|
|명칭|창업환경조성|창업기업지원융자(기금)|혁신창업사업화자금(융자)|


 2. 사업 지원 형태 및 지원율
|직접|출자|출연|보조|융자|국고보조율(%)|융자율 (%)|
|---|---|---|---|---|---|---|
||||○|○|||


 3. 지출계획 총괄표
|사업명|2022년 결산|2023년|Col4|2024년|증감 (B-A) (B-A)/A|Col7|
|---|---|---|---|---|---|---|
|||당초(A)|수정|확정(B)||(B-A)/A|
|혁신창업사업화자 금(융자)|2,300,000|2,230,000|2,330,000|2,007,800|△222,200|△9.96|


 4. 사업목적


(단위: 백만원, %)


ㅇ (창업기반지원) 기술력과 사업성이 우수하고 미래 성장가능성이 높은 중소벤처기업의
창업을 활성화하고 고용창출 도모
ㅇ (개발기술사업화) 중소기업이 보유한 우수 기술의 사장을 방지하고 개발기술의
제품화·사업화를 촉진하여 기술기반 중소기업을 육성
## 5. 사업근거 및 추진경위
ㅇ 법령상 근거 : 중소기업진흥에 관한 법률 제66조, 제67조, 제74조
중소기업창업지원법 제35조



{'title': '중소벤처기업부_혁신창업사업화자금(융자)'}
-----
ㅇ 추진경위
-  1998. 4 외환위기 이후 창업 활성화 및 고용 창출을 위

## 4.2 multi vector db

In [14]:
for db in pdf_databases_.keys():
    pdf_databases_[db]['retriever'] = pdf_databases_[db]['db'].as_retriever(search_kwargs={"k": 5})

In [15]:
pdf_databases_['중소벤처기업부_혁신창업사업화자금(융자)']['retriever'].get_relevant_documents("2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?")

[Document(metadata={'title': '중소벤처기업부_혁신창업사업화자금(융자)'}, page_content='## 사  업  명\n 혁신창업사업화자금(융자) (5152-301)\n\n\n## 1. 사업 코드 정보\n|구분|기금|소관|실국(기관)|계정|분야|부문|\n|---|---|---|---|---|---|---|\n|코드|중소벤처기업 창업 및 진흥기금|중소벤처기업부|중소기업정책실||110|118|\n|명칭|||||산업·중소기업 및 에너지|창업 및 벤처|\n\n\n|구분|프로그램|단위사업|세부사업|\n|---|---|---|---|\n|코드|5100|5152|301|\n|명칭|창업환경조성|창업기업지원융자(기금)|혁신창업사업화자금(융자)|\n\n\n 2. 사업 지원 형태 및 지원율\n|직접|출자|출연|보조|융자|국고보조율(%)|융자율 (%)|\n|---|---|---|---|---|---|---|\n||||○|○|||\n\n\n 3. 지출계획 총괄표\n|사업명|2022년 결산|2023년|Col4|2024년|증감 (B-A) (B-A)/A|Col7|\n|---|---|---|---|---|---|---|\n|||당초(A)|수정|확정(B)||(B-A)/A|\n|혁신창업사업화자 금(융자)|2,300,000|2,230,000|2,330,000|2,007,800|△222,200|△9.96|\n\n\n 4. 사업목적\n\n\n(단위: 백만원, %)\n\n\nㅇ (창업기반지원) 기술력과 사업성이 우수하고 미래 성장가능성이 높은 중소벤처기업의\n창업을 활성화하고 고용창출 도모\nㅇ (개발기술사업화) 중소기업이 보유한 우수 기술의 사장을 방지하고 개발기술의\n제품화·사업화를 촉진하여 기술기반 중소기업을 육성\n## 5. 사업근거 및 추진경위\nㅇ 법령상 근거 : 중소기업진흥에 관한 법률 제66조, 제67조, 제74조\n중소기업창업지원법 제35조'),
 Document(metadata={'title': '중소ᄇ

## 4.3 reranker

In [16]:
reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
compressor = CrossEncoderReranker(model=reranker_model, top_n=2)

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [17]:
for db in pdf_databases_.keys():
    pdf_databases_[db]['reranker'] = ContextualCompressionRetriever(
        base_compressor=compressor, base_retriever=pdf_databases_[db]['retriever']
    )

In [18]:
pdf_databases_['중소벤처기업부_혁신창업사업화자금(융자)']['reranker'].get_relevant_documents("2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?")

[Document(metadata={'title': '중소벤처기업부_혁신창업사업화자금(융자)'}, page_content='## 사  업  명\n 혁신창업사업화자금(융자) (5152-301)\n\n\n## 1. 사업 코드 정보\n|구분|기금|소관|실국(기관)|계정|분야|부문|\n|---|---|---|---|---|---|---|\n|코드|중소벤처기업 창업 및 진흥기금|중소벤처기업부|중소기업정책실||110|118|\n|명칭|||||산업·중소기업 및 에너지|창업 및 벤처|\n\n\n|구분|프로그램|단위사업|세부사업|\n|---|---|---|---|\n|코드|5100|5152|301|\n|명칭|창업환경조성|창업기업지원융자(기금)|혁신창업사업화자금(융자)|\n\n\n 2. 사업 지원 형태 및 지원율\n|직접|출자|출연|보조|융자|국고보조율(%)|융자율 (%)|\n|---|---|---|---|---|---|---|\n||||○|○|||\n\n\n 3. 지출계획 총괄표\n|사업명|2022년 결산|2023년|Col4|2024년|증감 (B-A) (B-A)/A|Col7|\n|---|---|---|---|---|---|---|\n|||당초(A)|수정|확정(B)||(B-A)/A|\n|혁신창업사업화자 금(융자)|2,300,000|2,230,000|2,330,000|2,007,800|△222,200|△9.96|\n\n\n 4. 사업목적\n\n\n(단위: 백만원, %)\n\n\nㅇ (창업기반지원) 기술력과 사업성이 우수하고 미래 성장가능성이 높은 중소벤처기업의\n창업을 활성화하고 고용창출 도모\nㅇ (개발기술사업화) 중소기업이 보유한 우수 기술의 사장을 방지하고 개발기술의\n제품화·사업화를 촉진하여 기술기반 중소기업을 육성\n## 5. 사업근거 및 추진경위\nㅇ 법령상 근거 : 중소기업진흥에 관한 법률 제66조, 제67조, 제74조\n중소기업창업지원법 제35조'),
 Document(metadata={'title': '중소ᄇ

In [13]:
pdf_databases_ = {k.replace(" v2", ""):v for k, v in pdf_databases_.items()}

# 5. load test data

In [14]:
import pandas as pd

test_df = pd.read_csv(project_path+'/open/test.csv')
test_df.head()

,SAMPLE_ID,Source,Source_path,Question
0,TEST_000,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
1,TEST_001,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?
2,TEST_002,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?
3,TEST_003,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?
4,TEST_004,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요?


In [15]:
test_df_ = test_df.groupby('Source').first().reset_index()
test_df_

,Source,SAMPLE_ID,Source_path,Question
0,「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》,TEST_043,./test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정...,국고보조사업의 보조율은 어떠한 기준에 따라 운용되는가?
1,「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》,TEST_046,./test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관...,국제적으로 성과중심 재정관리 강화 움직임이 확산된 시기는 언제인가?
2,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,TEST_078,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,우리나라에서는 언제부터 발생주의 기준을 적용한 국가결산보고서에서 우발부채를 보고하고...
3,「FIS 이슈&포커스」 22-2호 《재정성과관리제도》,TEST_057,./test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf,재정성과관리제도는 어떤 측면에서 국정운영과 연결되는가?
4,국토교통부_행복주택출자,TEST_038,./test_source/국토교통부_행복주택출자.pdf,행복주택출자 사업은 어떤 근거로 추진되고 있는가?
5,보건복지부_노인장기요양보험 사업운영,TEST_015,./test_source/보건복지부_노인장기요양보험 사업운영.pdf,노인장기요양보험 사업 운영에 대한 목적은 무엇인가요?
6,보건복지부_부모급여(영아수당) 지원,TEST_009,./test_source/보건복지부_부모급여(영아수당) 지원.pdf,부모급여 지원 사업의 목적은 무엇인가요?
7,산업통상자원부_에너지바우처,TEST_023,./test_source/산업통상자원부_에너지바우처.pdf,에너지 바우처 제도의 주요 내용은 무엇인가요?
8,중소벤처기업부_혁신창업사업화자금(융자),TEST_000,./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?


# 6. rag

In [19]:
import torch

device = torch.device("cuda:0")
print(f"Available memory on device {device.index}: {torch.cuda.mem_get_info(device)[0] / 1e9:.2f} GB")

Available memory on device 0: 39.65 GB


## 6.1 load Generator

In [21]:
from transformers import Gemma2ForCausalLM
import torch

def setup_llm_pipeline():
    # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # 모델 ID
    model_id = "rtzr/ko-gemma-2-9b-it"

    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = Gemma2ForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True )

    # Lora 적용
    adapter_name = "/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/pymupdf4llm/prompt_finetune/checkpoint-400"
    model = PeftModel.from_pretrained(model, adapter_name,is_trainable=False)

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        task="text-generation",
        return_full_text=False,
        max_new_tokens=512,
        repetition_penalty=1.0,
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return hf

hf = setup_llm_pipeline()

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json:   0%|          | 0.00/852 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00010.safetensors:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

...

model-00009-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00010-of-00010.safetensors:   0%|          | 0.00/705M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

In [22]:
print(hf("### 질문: 딥러닝이 뭐야?\n\n### 답변:"))

<ipython-input-22-5ae795f95e46>:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  print(hf("### 질문: 딥러닝이 뭐야?\n\n### 답변:"))


 딥러닝(Deep Learning)은 인공지능(AI) 분야에서 가장 뜨거운 주제 중 하나로, 인간의 뇌처럼 계층적 구조의 인공 신경망을 학습시켜 복잡한 데이터를 분석하고 예측하는 기술을 말해요.

**인간의 뇌를 모방한 구조**

딥러닝은 인간의 뇌의 신경망 구조를 모방하여 만들어진 인공 신경망을 학습시키는 방식을 말해요. 인공 신경망은 단순한 신경 세포인 뉴런을 모방한 '뉴런'과 뉴런들 사이의 연결을 의미하는 '가중치'로 구성되어 있어요. 이러한 뉴런들이 계층적으로 연결되어 정보를 전달하면서 복잡한 계산을 수행하는데, 이를 '층(Layer)'이라고 부르죠.

**단순한 뉴런의 연합이 복잡한 지식을 갖게 됨**

매우 단순한 뉴런이지만, 수많은 뉴런들이 계층적으로 연결되어 복잡한 계산을 수행할 수 있기 때문에 이미지 인식, 자연어 처리 등 인간이 수행하는 복잡한 작업을 딥러닝 모델이 학습할 수 있게 됨

**데이터를 통해 스스로 학습**

딥러닝 모델은 방대한 양의 데이터를 학습하여 스스로 특징을 찾고 지식을 습득하는 능력을 갖추고 있어요. 이를 통해 이미지 분류, 음성 인식, 자연어 처리 등 다양한 분야에서 뛰어난 성능을 보여주고 있습니다.

**주요 개념**

* **뉴런(Neuron):** 인공 신경망의 기본 단위로, 인간의 뇌의 뉴런과 같은 역할을 수행
* **가중치(Weight):** 뉴런들 사이의 연결의 강약을 나타내는 값으로, 학습 과정에서 조정
* **층(Layer):** 여러 뉴런들이 연결된 구조 단위로, 인공 신경망은 여러 층으로 구성
* **활성화 함수(Activation Function):** 뉴런의 출력


## 6.2 make chain & iterate test data

### 6.2.1 single vector db

In [42]:

def format_docs(docs):
    doc = "\n=====\n\n".join(doc.page_content for doc in docs) if len(docs) > 0 else ""
    return doc

template = """
다음 정보를 바탕으로 질문에 답하시오:
{context}

질문: {question}

주어진 질문에만 답하시오. 짧게 답하시오.
답변:
"""
prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    {"question": RunnablePassthrough(), "context": retriever | format_docs}
    | prompt
    | hf
    | StrOutputParser()
)


In [43]:
answer = rag_chain.invoke("중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?")
print(answer)

중소기업진흥에 관한 법률 제66조, 제67조, 제74조


In [44]:
from tqdm import tqdm

list_data = []
for _, row in tqdm(test_df_.iterrows()):
    qustion = row["Question"]
    answer = rag_chain.invoke(qustion)
    input_text = {'id': row["SAMPLE_ID"], 'answer': answer.strip()}
    list_data.append(input_text)
    print(_, input_text["answer"])

1it [00:14, 14.83s/it]

0 보조금법과 함께 일부 개별 법령에 근거하여 운용되며, 보조금법에 의해 지방 자치 단체의 재정여건을 고려하여 기준보조율과 차등보조율을 적용하여 운용하고 있다.


2it [02:08, 73.23s/it]

1 2000년대 후반 금융위기로 성과중심 재정관리 강화 움직임이 국제적으로 확산된 시기는 2007년 기준, 70% 이상의 OECD 국가에서 성과중심 재정관리 강화 움직임이 확산된 시기는 2007년 기준, 70% 이상의 OECD 국가에서 성과중심 재정관리 강화 움직임이 확산된 시기는 2007년 기준, 70% 이상의 OECD 국가에서 성과중심 재정관리 강화 움직임이 확산된 시기는 2007년 기준, 70% 이상의 OECD 국가에서 성과중심 재정관리 강화 움직임이 확산된 시기는 2007년 기준, 70% 이상의 OECD 국가에서 성과중심 재정관리 강화 움직임이 확산된 시기는 2007년 기준, 70% 이상의 OECD 국가에서 성과중심 재정관리 강화 움직임이 확산된 시기는 2007년 기준, 70% 이상의 OECD 국가에서 성과중심 재정관리 강화 움직임이 확산된 시기는 2007년 기준, 70% 이상의 OECD 국가에서 성과중심 재정관리 강화 움직임이 확산된 시기는 2007년 기준, 70% 이상의 OECD 국가에서 성과중심 재정관리 강화 움직임이 확산된 시기는 2007년 기준, 70% 이상의 OECD 국가에서 성과중심 재정관리 강화 움직임이 확산된 시기는 2007년 기준, 70% 이상의 OECD 국가에서 성과중심 재정관리 강화 움직임이 확산된 시기는 2007년 기준, 70% 이상의 OECD 국가에서 성과중심 재정관리 강화 움직임이 확산된 시기는 2007년 기


3it [02:25, 47.56s/it]

...

6 부모급여 지원 사업의 목적은 출산 및 양육으로 손실되는 소득을 보전하고, 주 양육자의 직접돌봄이 중요한 아동발달의 특성에 따라 영아기 돌봄을 두텁게 지원하기 위해 부모급여 지급을 목적으로 합니다.


8it [04:04, 24.28s/it]

7 에너지 바우처는 에너지 소비가 많은 저소득층을 대상으로 에너지 비용 부담을 경감해주는 제도입니다. 에너지 바우처는 에너지 소비가 많은 저소득층을 대상으로 에너지 비용 부담을 경감해주는 제도입니다. 에너지 바우처는 에너지 소비가 많은 저소득층을 대상으로 에너지 비용 부담을 경감해주는 제도입니다.


9it [04:13, 28.14s/it]

8 2022년 혁신창업사업화자금(융자)의 예산은 2,007,800백만원입니다.


### 6.3.2 multi vector db

In [23]:
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate

def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)

# Source 값에 따라 적절한 compression_retriever를 선택하는 함수
def choose_compression_retriever(source):
    return pdf_databases_[source]['reranker']

# RunnableLambda를 사용하여 source 값에 따라 리트리버 선택
compression_retriever_dynamic = RunnableLambda(
    lambda inputs: {"context": choose_compression_retriever(inputs["source"]), "question": inputs["question"]}
)

# 템플릿 구성
template = """
다음 정보를 바탕으로 질문에 답하세요:
{context}

질문: {question}

주어진 질문에만 답변하세요. 문장으로 짧게 답변해주세요.
답변:
"""
prompt = ChatPromptTemplate.from_template(template)

# 최종 체인 구성
rag_chain2 = (
    compression_retriever_dynamic
    | prompt
    | hf
    | StrOutputParser()
)

In [25]:
inputs = {"source": "중소벤처기업부_혁신창업사업화자금(융자)", "question": "2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?"}
answer = rag_chain2.invoke(inputs)
print(answer)

2022년 혁신창업사업화자금(융자)의 예산은 1,000억원입니다.


In [27]:
from tqdm import tqdm

sub_list = []
for _, row in tqdm(test_df_.iterrows(), total=len(test_df_), desc="Answering Questions"):
    inputs = {"source": unicodedata.normalize('NFC', row['Source']), "question": row['Question']}
    answer = rag_chain2.invoke(inputs).strip()
    sub_list.append(answer)

Answering Questions: 100%|██████████| 9/9 [01:22<00:00,  9.15s/it]


In [36]:
for (_, q), a in zip(test_df_.iterrows(), sub_list):
    print('=====================')
    print(f"출처: {q['Source']}\n질문: {q['Question']} \n답변: {a}")
    print('=====================\n')

출처: 「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》
질문: 국고보조사업의 보조율은 어떠한 기준에 따라 운용되는가? 
답변: 보조율은 사업의 목적과 범위, 사업추진체계, 사업추진방법 등에 따라 운용된다.

출처: 「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》
질문: 국제적으로 성과중심 재정관리 강화 움직임이 확산된 시기는 언제인가? 
답변: 국제적으로 성과중심 재정관리 강화 움직임이 확산된 시기는 1990년대 후반부터 2000년대 초반이다.

출처: 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》
질문: 우리나라에서는 언제부터 발생주의 기준을 적용한 국가결산보고서에서 우발부채를 보고하고 있는가? 
답변: 우리나라에서는 1994년부터 발생주의 기준을 적용한 국가결산보고서에서 우발부채를 보고하고 있다.

출처: 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》
질문: 재정성과관리제도는 어떤 측면에서 국정운영과 연결되는가? 
답변: 재정성과관리제도는 중장기 국가전략 수립과 연계하여 국정운영의 효율성을 제고하고, 재정운용의 투명성과 책임성을 강화하는 데 기여한다.

출처: 국토교통부_행복주택출자
질문: 행복주택출자 사업은 어떤 근거로 추진되고 있는가? 
답변: 행복주택출자사업은 주택공급 부족 현상에 대응하기 위해 추진되고 있다.

출처: 보건복지부_노인장기요양보험 사업운영
질문: 노인장기요양보험 사업 운영에 대한 목적은 무엇인가요? 
답변: 노인장기요양보험 사업은 노인의 장기요양필요에 신속하고 적절하게 대응하여 노인의 삶의 질 향상 및 사회복지 향상에 기여하는 것을 목적으로 운영되고 있습니다.

출처: 보건복지부_부모급여(영아수당) 지원
질문: 부모급여 지원 사업의 목적은 무엇인가요? 
답변: 부모급여 지원 사업의 목적은 출산·입양 부모가 자녀 양육에 집중할 수 있도록 경제적 부담을 경감하는 것입니다.

출처: 산업통상자원부_에너지바우처
질문: 에너지 바우처 제도의 주요 내용은 무엇인가요? 
답변: 에너지 바우처 제

## 6.3 save inferenced test data

In [ ]:
test = pd.DataFrame(list_data)
test.columns = ['SAMPLE_ID', 'Answer']
test

,SAMPLE_ID,Answer
0,TEST_000,"2,300,000 백만원"
1,TEST_001,"창업기반지원, 개발기술사업화를 통해 중소벤처기업의 창업 활성화 및 고용 창출, 기술..."
2,TEST_002,"중소기업진흥에 관한 법률 제66조, 제66조, 제67조, 제74조"
3,TEST_003,재창업자금(실패 경영인에 대한 재기지원)
4,TEST_004,"미래기술육성자금, 고성장촉진자금"
...,...,...
93,TEST_093,공적보증채무는 공공부문 단위가 계약에 의해 지급을 보증하는 공적보증채무(public...
94,TEST_094,현행법과 규정에 따라 이미 가입자가 획득한 미래 급여의 현재가치에서 현행법과 규정에...
95,TEST_095,국가결산보고서는 우발부채를 재무제표상 부채로 인식하지 않고 주석에 공시합니다. 반면...
96,TEST_096,"우발부채는 미래에 발생이 불확실한 잠재적 의무 또는 현재의무로, 미래의 다양한 의제..."


In [ ]:
test.to_csv("/content/drive/MyDrive/Data/dacon/finance/results/"+"ko-gemma-2-9b-it_2.csv", index=False)

In [ ]:
import chardet

RECIPE_FILE = "/content/drive/MyDrive/Data/dacon/finance/results/"+"ko-gemma-2-9b-it_2.csv"
rawdata = open(RECIPE_FILE, 'rb').read()
result = chardet.detect(rawdata)
charenc = result['encoding']

In [ ]:
charenc

'utf-8'